In [7]:
import pandas as pd
from py2neo import Graph
import numpy as np
import itertools

In [8]:
#py2neo连接图数据库
lib_graph = Graph(
    "http://localhost:7474",
    username="neo4j",
    password="neo4j"
)

## 个人主页推荐-两种策略结果混合

In [9]:
def getRec1(name1):
    name = name1
    # 同作者的其他图书推荐
    df1 = lib_graph.run(
        "match (a:Reader)-[r]->(b:BOOK)<-[:write]-(c:Author)-[:write]->(d:BOOK) where a.NAME={param} return d.ISBN, d.TITLE, d.TOTAL_CIRC order by d.TOTAL_CIRC desc",
        param=name).to_data_frame()
    df1 = df1.drop_duplicates()
    person_df1 = df1.rename(columns={'d.ISBN': 'ISBN', 'd.TITLE': 'TITLE', 'd.TOTAL_CIRC': 'TOTAL_CIRC'}, index=None)
    person_df1.sort_values(by='TOTAL_CIRC', ascending=False) # 降序排序
    # 找到所有和目标人物读过至少两本不同书的人
    df_people = lib_graph.run(
        "match p = (a:Reader)-[r1]->(b1:BOOK)<-[r2]-(c:Reader)-[r3]->(b2:BOOK)<-[r4]-(a) where a.NAME = {param} and b1 <> b2 return distinct c.CERT_ID, c.NAME",
        param=name).to_data_frame()
    people_list = np.array(df_people['c.CERT_ID']).tolist()
    people_list2 = list(itertools.combinations(people_list, 2)) # 排列组合
    # 循环找到兴趣相投的人共同读过的所有书目
    rec_list = pd.DataFrame(columns=['b1.ISBN', 'b1.TITLE'])
    for item in people_list2:
        param1 = item[0]
        param2 = item[1]
        data1 = lib_graph.run(
            "match p = (a:Reader)-[r1]->(b1:BOOK)<-[r2]-(b:Reader) where b.CERT_ID={people1} and a.CERT_ID={people2} return b1.TITLE, b1.ISBN, b1.TOTAL_CIRC",
            people1=param1, people2=param2).to_data_frame()
        if data1.empty == False:
            result = rec_list.append(data1)
            rec_list = result
    # 共同兴趣的人的推荐列表
    df2 = rec_list.drop_duplicates()
    person_df2 = df2.rename(columns={'b1.ISBN': 'ISBN', 'b1.TITLE': 'TITLE', 'b1.TOTAL_CIRC': 'TOTAL_CIRC'})
    person_df2.sort_values(by='TOTAL_CIRC', ascending=False) # 降序排序
    # 综合两种策略的推荐结果
    result1 = pd.concat([person_df1[:5], person_df2[:5]])
    result1 = result1.sort_values(by='TOTAL_CIRC', ascending=False)
    result1.index = range(len(result1))
    return result1

In [10]:
if __name__ == '__main__':
    name1 = '殷要'                  #按照学号更合理
    res = getRec1(name1)
    print(res)

C:\Users\Anaconda3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


                ISBN                  TITLE  TOTAL_CIRC
0      7-80179-116-9                   与我同行       473.0
1      7-5442-2550-X                    长恨歌       392.0
2      7-5313-2437-7                     流逝       355.0
3      7-5635-1367-1               人工神经网络教程       277.0
4  978-7-121-04089-4  神经网络理论与MATLAB R2007实现       248.0
5      7-5321-2621-8                   桃之夭夭       245.0
6      7-5605-1448-0        遗传算法:理论、应用与软件实现       243.0
7  978-7-5025-9523-4         人工神经网络理论、设计及应用       225.0
8  978-7-5170-4066-8              随机水文学.第3版        10.0
9  978-7-114-12510-2         突发事件条件下的列车运行组织         5.0


In [11]:
if __name__ == '__main__':
    name1 = '谢泽宇'                  #按照学号更合理
    res = getRec1(name1)
    print(res)

C:\Users\Anaconda3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


                ISBN                             TITLE  TOTAL_CIRC
0  978-7-121-07934-4  HTML/CSS/Java Script标准教程:实例版.第2版        93.0
1      7-302-12505-8                        复杂网络理论及其应用        74.0
2  978-7-115-31398-0                           SQL必知必会        44.0
3      7-5008-3161-7                                沉浮        20.0
4  978-7-03-034599-8                          复杂网络基础理论        13.0
5  978-7-115-39439-2                    Python参考手册.第2版        13.0
6  978-7-03-023763-7                      网络信息分类:原理与应用        10.0
7  978-7-3012-5553-7                            空间数据分析         6.0
8  978-7-03-027184-6                        时空数据库查询与推理         5.0
9  978-7-115-43921-5                 Oracle PL/SQL必知必会         4.0


## 策略1-同作者其他图书推荐 

In [12]:
def getRec_Author(name1):
    name = name1
    # 同作者的其他图书推荐
    df1 = lib_graph.run(
        "match (a:Reader)-[r]->(b:BOOK)<-[:write]-(c:Author)-[:write]->(d:BOOK) where a.NAME={param} return d.ISBN, d.TITLE, d.TOTAL_CIRC order by d.TOTAL_CIRC desc",
        param=name).to_data_frame()
    df1 = df1.drop_duplicates()
    person_df1 = df1.rename(columns={'d.ISBN': 'ISBN', 'd.TITLE': 'TITLE', 'd.TOTAL_CIRC': 'TOTAL_CIRC'}, index=None)
    person_df1.sort_values(by='TOTAL_CIRC', ascending=False) # 降序排序
    person_df1.index = range(len(person_df1))
    return person_df1

In [13]:
if __name__ == '__main__':
    name1 = '潘奕慧'
    res = getRec_Author(name1)
    print(res)

                ISBN                                              TITLE  \
0  978-7-5118-3582-6                                            国际河流法研究   
1  978-7-5097-6450-3  水霸权、安全秩序与制度构建:国际河流水政治复合体研究:on international ri...   

   TOTAL_CIRC  
0           8  
1           3  


## 策略2-相近兴趣的其他读者的共同阅读书籍 

In [14]:
def getRec_Friend(name1):
    name = name1
    # 找到所有和目标人物读过至少两本不同书的人
    df_people = lib_graph.run(
        "match p = (a:Reader)-[r1]->(b1:BOOK)<-[r2]-(c:Reader)-[r3]->(b2:BOOK)<-[r4]-(a) where a.NAME = {param} and b1 <> b2 return distinct c.CERT_ID, c.NAME",
        param=name).to_data_frame()
    people_list = np.array(df_people['c.CERT_ID']).tolist()
    people_list2 = list(itertools.combinations(people_list, 2)) # 排列组合
    # 循环找到兴趣相投的人共同读过的所有书目
    rec_list = pd.DataFrame(columns=['b1.ISBN', 'b1.TITLE'])
    for item in people_list2:
        param1 = item[0]
        param2 = item[1]
        data1 = lib_graph.run(
            "match p = (a:Reader)-[r1]->(b1:BOOK)<-[r2]-(b:Reader) where b.CERT_ID={people1} and a.CERT_ID={people2} return b1.TITLE, b1.ISBN, b1.TOTAL_CIRC",
            people1=param1, people2=param2).to_data_frame()
        if data1.empty == False:
            result = rec_list.append(data1)
            rec_list = result
    # 共同兴趣的人的推荐列表
    df2 = rec_list.drop_duplicates()
    person_df2 = df2.rename(columns={'b1.ISBN': 'ISBN', 'b1.TITLE': 'TITLE', 'b1.TOTAL_CIRC': 'TOTAL_CIRC'})
    person_df2.sort_values(by='TOTAL_CIRC', ascending=False) # 降序排序
    person_df2.index = range(len(person_df2))
    return person_df2

In [15]:
if __name__ == '__main__':
    name1 = '谢泽宇'
    res = getRec_Friend(name1)
    print(res)

                ISBN                                              TITLE  \
0  978-7-121-07934-4                   HTML/CSS/Java Script标准教程:实例版.第2版   
1  978-7-03-023763-7                                       网络信息分类:原理与应用   
2      7-302-12505-8                                         复杂网络理论及其应用   
3  978-7-03-034599-8                                           复杂网络基础理论   
4  978-7-115-39439-2                                     Python参考手册.第2版   
5  978-7-121-27648-4  大数据智能:互联网时代的机器学习和自然语言处理技术:machine learning & n...   
6      7-302-05234-4                                       Java技术教程.基础篇   

   TOTAL_CIRC  
0        93.0  
1        10.0  
2        74.0  
3        13.0  
4        13.0  
5         4.0  
6       154.0  


C:\Users\Anaconda3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [16]:
def getRec_Friend_1(name1):
    name = name1
    # 找到所有和目标人物读过至少两本不同书的人
    df_people = lib_graph.run(
        "match p = (a:Reader)-[r1]->(b1:BOOK)<-[r2]-(c:Reader) where a.NAME = {param} return distinct c.CERT_ID, c.NAME",
        param=name).to_data_frame()
    people_list = np.array(df_people['c.CERT_ID']).tolist()[:10]    # 10 是自己设定的超参数
    people_list2 = list(itertools.combinations(people_list, 2)) # 排列组合
    # 循环找到兴趣相投的人共同读过的所有书目
    rec_list = pd.DataFrame(columns=['b1.ISBN', 'b1.TITLE'])
    for item in people_list2:
        param1 = item[0]
        param2 = item[1]
        data1 = lib_graph.run(
            "match p = (a:Reader)-[r1]->(b1:BOOK)<-[r2]-(b:Reader) where b.CERT_ID={people1} and a.CERT_ID={people2} return b1.TITLE, b1.ISBN, b1.TOTAL_CIRC",
            people1=param1, people2=param2).to_data_frame()
        if data1.empty == False:
            result = rec_list.append(data1)
            rec_list = result
    # 共同兴趣的人的推荐列表
    df2 = rec_list.drop_duplicates()
    person_df2 = df2.rename(columns={'b1.ISBN': 'ISBN', 'b1.TITLE': 'TITLE', 'b1.TOTAL_CIRC': 'TOTAL_CIRC'})
    person_df2.sort_values(by='TOTAL_CIRC', ascending=False) # 降序排序
    person_df2.index = range(len(person_df2))
    return person_df2

In [17]:
if __name__ == '__main__':
    name1 = '潘奕慧'
    res = getRec_Friend_1(name1)
    print(res)

C:\Users\Anaconda3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


                ISBN             TITLE  TOTAL_CIRC
0  978-7-5084-7997-2      国际水资源规划及水利工程        12.0
1      7-5630-0838-1           水工建筑物.上       957.0
2      7-5611-2750-2          雅思8分万能作文       141.0
3  978-7-5084-6873-0        国外流域综合规划技术        29.0
4  978-7-111-24481-3  HyperWorks分析应用实例        72.0


有的读者没有共同阅读过两本书籍的相近兴趣的人，所以需要根据共同阅读一本的书籍进行分析